# Parcial 2 
- Alumno: Juan Cruz Guglieri
- DNI: 44329939

In [85]:
from random import random, choice
import numpy as np
from scipy.stats import chi2, norm
import math

## Ejercicio 2

In [6]:
# Los datos del ejercicio:
data = [15.22860536, 40.60145536, 33.67482894, 44.03841737, 15.69560109, 16.2321714, 25.02174735, 30.34655637, 3.3181228, 5.69447539, 10.1119561, 49.10266584, 3.6536329, 35.82047148, 3.37816632, 36.72299321, 50.67085322, 3.25476304, 20.12426236, 20.2668814, 17.49593589, 2.70768636, 14.77332745, 1.72267967, 23.34685662, 8.46376635, 9.18330789, 9.97428217, 2.33951729, 137.51657441, 9.79485269, 10.40308179, 1.57849658, 6.26959703, 4.74251574, 1.53479053, 34.74136011, 27.47600572, 9.1075566, 1.88056595, 27.59551348, 6.82283137, 12.45162807, 28.01983651, 0.36890593, 7.82520791, 3.17626161, 46.91791271, 38.08371186, 41.10961135]

In [11]:
def F_exponencial(x, lambda_):
    return 1 - np.exp(-x*lambda_)

n = len(data)

# Kolmogorov-Smirnov con exponenciales
def K_S_exponencial(datos, lambda_):
    datos.sort()
    d = 0
    for j in range(n):
        x = datos[j]
        F_x = F_exponencial(x, lambda_)
        # D = max_{1<=j<=n} {j/n - F(X_j), F(X_j) - (j-1)/n}
        d = max(d, (j+1)/n - F_x, F_x-j/n)
    return d

d = K_S_exponencial(data, 0.05)
print("Estadistico D:", d)

Estadistico D: 0.0744289456538767


## c)  
**Determine si la hipótesis nula es rechazada o no, con un nivel de rechazo del 4%. Para esto, utilizar simulaciones con variables uniformes.**

In [35]:
# Simulamos el p-valor usando variables uniformes
def sim(Nsim=1000):
    p_valor = 0
    for _ in range(Nsim):
        # Generamos n variables uniformes
        uniformes = np.random.uniform(0, 1, n)
        exp_sim = -np.log(1 - uniformes) / 0.05
        d_j = K_S_exponencial(exp_sim.tolist(), 0.05)
        if d_j >= d:
            p_valor += 1
    return p_valor / Nsim

p_valor = sim()
print("p-valor simulado=", p_valor)

if p_valor < 0.04:
    print(f"Como {p_valor*100}% < 4%; Hipotesis nula rechazada")
else:
    print(f"Como {p_valor*100}% > 4%; Hipotesis nula no es rechazada")


p-valor simulado= 0.913
Como 91.3% > 4%; Hipotesis nula no es rechazada


## d)
**Determine si la hipótesis nula es rechazada o no, con un nivel de rechazo del 4%, esta vez simulando variables que verifiquen la hipótesis nula.**

In [33]:
# Simulamos el p-valor usando variables exponenciales con lambda 0.05
def sim(Nsim=1000):
    p_valor = 0
    for _ in range(Nsim):
        # Generamos n variables exponenciales con lambda 0.05
        exp_sim = np.random.exponential(scale=1/0.05, size=n)
        d_j = K_S_exponencial(exp_sim.tolist(), 0.05)
        if d_j >= d:
            p_valor += 1
    return p_valor / Nsim

p_valor = sim()
print("p-valor simulado=", p_valor)

if p_valor < 0.04:
    print(f"Como {p_valor*100}% < 4%; Hipotesis nula rechazada")
else:
    print(f"Como {p_valor*100}% > 4%; Hipotesis nula no es rechazada")

p-valor simulado= 0.92
Como 92.0% > 4%; Hipotesis nula no es rechazada


## Ejercicio 3

In [ ]:
frec_obs = [38, 144, 342, 287, 164, 25]
n = sum(frec_obs)
k = len(frec_obs) # Cantidad de intervalos o número de agrupamiento de los valores 

# Calculo de p
E_X_obs = sum(i * frec_obs[i] for i in range(6))
p_est = (E_X_obs / 5)  / n  # Estimación de p para una Binomial(5, p)
print("Parametro p estimado:", p_est)

# Probabilidades teoricas p_i
probs = []
for i in range(6):
    comb = math.comb(5, i)
    probs.append(comb * p_est**i * (1-p_est)**(5-i))


# Estadístico de prueba
T = 0
for i in range(k):
    frec_esp = n * probs[i]
    T += (frec_obs[i] - frec_esp)**2 / frec_esp
    
print("T de Pearson:", T)
gl = k - 1 - 1 # Grados de libertad
p_valor = chi2.sf(T, gl)
print(f"p-valor: {p_valor:.5f}")

Parametro p estimado: 0.494
T de Pearson: 8.161627432060547
p-valor: 0.08583


## c)

In [82]:
def sim(Nsim=1000):
    p_valor = 0
    for _ in range(Nsim):
        
        # Genero la frecuencia muestral con una binomial, con n=5, y p=p_est=0.494
        frec_muestral = [0] * 6
        muestras = np.random.binomial(5, p_est, 1000)
        for i in muestras:
            frec_muestral[i] += 1 
            
        # Estimacion de p con los datos nuevos:
        E_X_sim = sum(i * frec_muestral[i] for i in range(6))
        p_est_sim = (E_X_sim / 5)  / n 
        
        # Probabilidades teoricas p_i
        probs_sim = []
        for i in range(6):
            comb = math.comb(5, i)
            probs_sim.append(comb * p_est_sim**i * (1-p_est_sim)**(5-i))
        
        # Calculo de t_sim
        t_sim = 0
        for s in range(6):
            frec_esp_sim = probs_sim[s] * n
            t_sim += (frec_muestral[s] - frec_esp_sim)**2 / frec_esp_sim
        
        if t_sim >= T:
            p_valor += 1
            
    return p_valor / Nsim
            
print("p-valor simulado:", sim())

p-valor simulado: 0.088


## Ejercicio 4

In [ ]:
def monteCarlo_0_a_1(g, Nsim):
    integral = 0
    for _ in range(Nsim):
        integral += g(random())
    return integral / Nsim


def g(u):
    return np.exp(-(u+2)) * (1 - (u + 2)**4)

print("Media por montecarlo:", monteCarlo_0_a_1(g, 100000)) 
print()
print("Media por estimacion por intervalo:")


def Media_Muestral_X(g, z_alpha_2, L):
    d = L / (2 * z_alpha_2)
    media = g(random())
    Scuad = 0
    n = 1
    while n <= 100 or np.sqrt(Scuad / n) > d:
        n += 1
        X = g(random())
        media_ant = media
        media = media_ant + (X - media_ant) / n
        Scuad = Scuad * (1 - 1 / (n - 1)) + n * (media - media_ant) ** 2
        if n==1000 or n==5000 or n==7000:
            S = np.sqrt(Scuad)
            IC = (media - z_alpha_2 * S / np.sqrt(n), media + z_alpha_2 * S / np.sqrt(n))
            print(f"N° de sim: {n}")
            print(f"I: {media:.4f}")
            print(f"S: {S:.4f}")
            print(f"IC: ({IC[0]:.4f}, {IC[1]:.4f})\n")
            
        
    S = np.sqrt(Scuad)
    IC = (media - z_alpha_2 * np.sqrt(Scuad) / np.sqrt(n), media + z_alpha_2 * np.sqrt(Scuad) / np.sqrt(n))
    return n, media, S, IC

alpha = 0.95
z_alpha_2 = norm.ppf(1 - (1 - alpha) / 2) # = 1.96
d = 0.001
L = 2 * d # = 0.002 
n, I, S, IC = Media_Muestral_X(g=g, z_alpha_2=z_alpha_2, L=L)
print(f"N° de sim: {n}")
print(f"I: {I:.4f}")
print(f"S: {S:.4f}")
print(f"IC: ({IC[0]:.4f}, {IC[1]:.4f})")

Media por montecarlo: -3.081026686859957

Media por estimacion por intervalo:
N° de sim: 1000
I: -3.1045
S: 0.5595
IC: (-3.1391, -3.0698)

N° de sim: 5000
I: -3.0969
S: 0.5675
IC: (-3.1126, -3.0812)

N° de sim: 7000
I: -3.0962
S: 0.5665
IC: (-3.1094, -3.0829)

N° de sim: 1251582
I: -3.0849
S: 0.5708
IC: (-3.0859, -3.0839)
